In [1]:
import torch
import torch.nn as nn
from models import SRCNN
from Gloss import GradientVariance
from torch.utils.data.dataloader import DataLoader
from datasets import TrainDataset, EvalDataset
from tqdm import tqdm
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model = SRCNN()
model.load_state_dict(torch.load('./DLAM_weights/srcnn_x3.pth', map_location=device),strict=False)
model.to(device)

C:\Users\nicla\AppData\Local\Temp\ipykernel_16724\3535356963.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('./DLAM_weights/srcnn_x3.p

SRCNN(
  (conv1): Conv2d(1, 64, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
  (conv2): Conv2d(64, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv3): Conv2d(32, 1, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu): ReLU()
)

In [2]:
params = sum(p.numel() for p in model.parameters())
params

57281

In [3]:
patch_size = 8
gradloss_weight  = 0.01


lr = 1e-4
epochs = 1
batch_size = 32
num_workers = 8

train_file = "./91_Image_train/91-image_x4.h5"
eval_file = "./Set_5_eval/Set5_x4.h5"


criterion = nn.MSELoss().to(device)
grad_criterion = GradientVariance(patch_size=8).to(device)
optimizer = torch.optim.Adam([
    {'params': model.conv1.parameters()},
    {'params': model.conv2.parameters()},
    {'params': model.conv3.parameters(), 'lr': lr * 0.1}
], lr=lr)

train_dataset = TrainDataset(train_file)
train_dataloader = DataLoader(dataset=train_dataset,
                                batch_size=batch_size,
                                shuffle=True,
                                num_workers=num_workers,
                                pin_memory=True,
                                drop_last=True)
eval_dataset = EvalDataset(eval_file)
eval_dataloader = DataLoader(dataset=eval_dataset, batch_size=1)

In [27]:
for i in range(epochs):
    model.train()
    for data in tqdm(train_dataloader):
            inputs, labels = data

            inputs = inputs.to(device)
            labels = labels.to(device)

            preds = model(inputs)

            loss_mse = criterion(preds, labels)

            loss_grad = grad_criterion(preds, labels)

            loss = loss_mse + loss_grad

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            break


  0%|          | 0/680 [00:00<?, ?it/s]

GRAYS 2
torch.Size([32, 1, 33, 33])
torch.Size([32, 1, 33, 33])


  0%|          | 0/680 [00:02<?, ?it/s]


In [3]:
import pandas as pd

df = pd.read_csv(("DLAM_weights/srcnn_x4.csv"), names=['epoch', 'train_loss', 'eval_loss', 'psnr'])

df.head()

,epoch,train_loss,eval_loss
0,1,0.005479,0.001960
1,1,0.006082,0.001971
2,2,0.002419,0.001855


In [7]:
model = SRCNN().to(device)

state_dict = model.state_dict()
test_path = "test/x3/best.pth"
weight_path = "DLAM_weights/srcnn_x3.pth"
for n, p in torch.load(weight_path, map_location=lambda storage, loc: storage).items():
    if n == "model_state_dict":
        for n1, p1 in p.items():
            if n1 in state_dict.keys():
                state_dict[n1].copy_(p1)
                print(n1)
    # if n in state_dict.keys():
    #     state_dict[n].copy_(p)
    #     print(n)
    #     print(n)
    # else:
    #     continue

model.eval()

C:\Users\nicla\AppData\Local\Temp\ipykernel_16724\2048993444.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  for n, p in torch.load(weight_path, map_location=lambda stor

SRCNN(
  (conv1): Conv2d(1, 64, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
  (conv2): Conv2d(64, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv3): Conv2d(32, 1, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu): ReLU()
)